**AN INTRODUCTION TO NEURAL NETWORKS**

*Patrick Donnelly*

This lab introduces the basic operations for training neural networks. We examine matrix multiplications and additions, convolutions, and softmax, cross-entropy, and activation functions in pure Python, NumPy, and PyTorch. By the end of this lab, you will be able to code a neural network for image classification in PyTorch and understand the mathematics behind deep learning.

We can think of the simplest network as consisting of an **input node** and an **output node**.

We can then apply a **weight** to the input to generate the output. The weight multiplies the input `x` by a constant `w` to generate the output. If `w=1`, the output is simply the input.

Let's build a Python function `output` that returns the product of an input `x` and a weight `w`:

In [1]:
def output (x, w):
    return w*x

What does `output` return when `x=1` and `w=1`?

In [2]:
output (1, 1)

1

We can also add a **bias**, or constant value `b` added to the input

By convention, we represent biases as nodes

If `b=1`, we add one to our weighted input to generate our output

Let's build another Python function `output_with_bias` that takes our input `x`, a weight `w`, and a bias `b` and computes the output:

In [3]:
def output_with_bias (x, w, b):
    return w*x + b

What does `output` return when `x=1` and `b=1`? 

In [4]:
output_with_bias (x=1, w=1, b=1)

2

We've now defined a line. However, when training a neural network, we don't know the values of `w` or `b`. We need to use our inputs, outputs, and a **learning algorithm** to learn `w` and `b`.

Let's say we have one input/output pair, and our output is equal to $2 * input + 1$. What is our output when our input is $0$?

In [5]:
output_with_bias (x=0, w=2, b=1)

1

However, when training a model, we don't "know" `w` or `b`, and there are an infinite number of pairs of $(w, b)$ that satisfy $f(0) = 1$. Make sense?

With two input/output pairs, we can solve for our weight and bias. Let's write a function `linear_solver` that takes two inputs and two outputs and returns a weight and bias that fits both points:

In [6]:
def linear_solver (x1, y1, x2, y2):
    w = (y2 - y1)/(x2 - x1)
    b = y1 - w*x1
    return "w={}".format(w), "b={}".format(b)

What's our learned weight?

In [7]:
print(linear_solver (0, 1, 1, 3)[0])

w=2.0


What about the bias?

In [8]:
print(linear_solver (0, 1, 1, 3)[1])

b=1.0


However, this isn't how neural networks work. Neural networks (NNs) don't solve linear models analytically. NNs use something called **backpropagation**, modifying the value of the weights and biases to minimize the discrepancy between actual and predicted outputs. This is super inefficient in our example, but don't worry about that for now.

Let's go back to our example with $(x,y) = (0,1)$. Our **loss function**, $L(y, Y)$ tells us how well our network is performing. The loss function measures the discrepancy between actual loss $y$ and predicted loss $Y$.

Let's start with an obvious loss function: absolute difference, where $L(y, Y) = |y - Y|$. What does `absolute_difference` look like in Python?

In [9]:
def absolute_difference (y, Y):
    return abs(y - Y)

Now we need initial values of `w` and `b` to generate predicted outputs. We'll start with zero initialization (which is generally a terrible idea, but that's okay for now). What's our output (with bias) when $(w, b) = (0, 0)$? Let's call our output `Y` and then print the value of `Y`:

In [10]:
Y = output_with_bias (0, 0, 0)
print(Y)

0


We see that our input $0$ will generate output $0$, and thus our loss is $|1-0| = |1| = 1$. However, the "true" value of $y$ is generated (deterministically) from $y = 2x + 1$. What's the "true" value of our output when our input is $0$?

In [11]:
y = output_with_bias (0, 2, 1)
print(Y)

0


Now let's compute our loss from our actual output `y` and predicted output `Y`:

In [12]:
absolute_difference(y, Y)

1

Once we calculate our loss, we need to update our weight and bias. We update our "network" by taking the derivatives of the loss function with respect to our weight and bias. We call these derivatives the **gradients** of our error function with respect to our weight and bias.

However, there's a problem with our loss function $L(x)$: it's not differentiable when $|y - Y| = 0$. Think about the absolute value function: it's $f(x) = -x$ when $x < 0$, $f(x) = x$ when $x > 0$, and $f(x) = 0$ when $x = 0$.

So for $f(x) = |x|$, $f'(x) = -1$ when $x < 0$ and $f'(x) = 1$ when $x > 0$. But at $f'(x) = 0$ we can't really tell if the function is going up or down; it's kind of a "hard inflection point" (is that a mathematically precise term?)

We need a loss function that is continuously differentiable. Why not just square $|y - Y|$?. Now $L(y, Y) = (y - Y)^2$. What does this look like in code?

In [13]:
def squared_difference (y, Y):
    return (y - Y)**2

What value does this function return when `y=1` and `Y=0`?

In [14]:
squared_difference(y=1, Y=0)

1

Now let's differentiate $L(y, Y) = (y - Y)^2$ with respect to our weight $w$

For $w$,

$\partial L(x)/\partial w = \partial/\partial w (y - Y)^2 =$

$\partial/\partial w (y - (wx + b))^2 =$

$\partial/\partial w (y - wx - b)^2 =$

$2(y - wx - b)(-x) =$

$-2x(y - wx - b)$

We can use this to build a function `dL_dW` to compute $\partial L(x)/\partial w$. Since $L(y, Y)$ is a function of our actual (ground truth) output $y$ and model (prediction) output $Y$, and $Y = wx + b$, we can use the input to our loss function $L(y, Y) = L(y, (x, (w, x, b))) = L(x, y, w, b)$ to update our weight (and subsequently bias). How do we code `dL_dW`?

In [15]:
def dL_dw (x, y, w, b):
    return -2*x*(y - w*x - b)

For our example with $(x, y) = (0, 1)$, and zero initialization: $(w, b) = (0, 0)$, compute $\partial L(y, Y)/ \partial w$:

In [16]:
dL_dw (0, 1, 0, 0)

0

Let's also differentiate $L(x) = (y - Y)^2$ with respect to our bias $b$

For $b$,

$\partial L(x)/\partial b = \partial/\partial$b $(y - Y)^2 =$

$\partial/\partial b (y - (wx + b))^2 =$

$\partial/\partial b (y - wx - b)^2 =$

$2(y - wx - b)(-1) =$

$-2(y - wx - b)$

Just as we did with our weight $w$, we can use this to build a function `dL_dB` to compute $\partial L(y, Y)/\partial b$. How do we code this function?

In [17]:
def dL_db (x, y, w, b):
    return -2*(y - w*x - b)

For the same example with $(x, y) = (0, 1)$, and zero initialization: $(w, b) = (0, 0)$, compute $\partial L(y, Y)/\partial b$:

In [18]:
dL_db (0, 1, 0, 0)

-2

Cool, let's now use these gradients to update our weights and biases. We'll use something called **gradient descent**:

$w_{t+1} = w_{t} - \alpha (\partial L(y, Y)/\partial w_{t})$

Here we are updating our weight from $w_{t}$ to $w_{t+1}$ by subtracting the scaled gradient of the loss function with respect to the weight. We scale the gradient by something called the **learning rate**, conventionally represented by $\alpha$

In the next block of code, we'll augment our `dL_dw` function to update the weight from $w_{t}$ to $w_{t+1}$. Let's call this function `gd_weight`:

In [19]:
def gd_weight (x, y, w, b, a):
    w = w - a * (-2*x*(y - w*x - b))
    return "w={}".format(w)

Let's use the same example $(x, y) = (0, 1)$ with zero initialization $(w, b) = (0, 0)$, and learning rate $a = 0.1$:

In [20]:
print(gd_weight (0, 1, 0, 0, 0.1))

w=0.0


So our weights don't actually update when $x = 0$ and $L(y, Y)$ is the squared error function $(y - Y)^2$. We'll need another example to update the weights. However, we can use this example to update the bias $b_{t}$ to $b_{t+1}$:

$b_{t+1}$ = $b_{t}$ - $\alpha$ ($\partial$E(y, Y)/$\partial$$b_{t}$)

How about coding this in Python? We'll call it `gd_bias`:

In [21]:
def gd_bias (x, y, w, b, a):
    b = b - a * (-2*(y - w*x - b))
    return "b={}".format(b)

What's the output of `gd_bias` when $(x, y) = (0, 1)$, assuming zero initialization of weights and biases and a learning rate $a = 0.1$?

In [22]:
print(gd_bias(x=0, y=1, w=0, b=0, a=0.1))

b=0.2


We can now feed a second datum generated from the line $y = 2x + 1$. When $x = 1$, $y = 2(1) + 1 = 3$. We use this to compute our new output $Y$. Let's set our predicted output `Y` equal to this output and print the output. Recall that our function is called `output_with_bias`:

In [23]:
Y = output_with_bias (x=1, w=0, b=0.2)
print(Y)

0.2


Our loss for this example is the squared difference between $y=3$ and $Y=0.2$. Let's call `squared_difference` using these two arguments:

In [24]:
squared_difference(y=3, Y=0.2)

7.839999999999999

This might look worse than when $y=1$ and $Y=0$. However, consider if we had fed the $(1, 3)$ example first. What would be our loss in this case?

In [25]:
Y = output_with_bias (x=1, w=0, b=0)
squared_difference(y=3, Y=Y)

9

So our loss is decreasing, and if we update our weight and bias again, we'll see our loss continue to decrease. Let's first update our weight and print the value of our new weight:

In [26]:
print(gd_weight (x=1, y=3, w=0, b=0.2, a=0.1))

w=0.5599999999999999


Now let's update our bias (using the weight before our update):

In [27]:
print(gd_bias (x=1, y=3, w=0, b=0.2, a=0.1))

b=0.76


Let's do one more "forward pass" with $(x, y) = (0, 1)$ and the weights and biases updated to $0.56$ and $0$:

In [28]:
Y = output_with_bias (x=0, w=0.56, b=0.76)
Y

0.76

Now we can see the loss continue to decrease, not only compared to the loss computed for $(1, 3)$, but also compared to the prior weights and biases applied to $(0, 1)$:

In [29]:
print(squared_difference(y=1, Y=output_with_bias (x=0, w=0, b=0)))
print(squared_difference(y=1, Y=output_with_bias (x=0, w=0, b=0.2)))
print(squared_difference(y=1, Y=output_with_bias (x=0, w=0.56, b=0.76)))             

1
0.6400000000000001
0.0576


Let's do one final weight and bias update:

In [30]:
print(gd_weight (x=0, y=1, w=0.56, b=0.76, a=0.1))
print(gd_bias (x=0, y=1, w=0.56, b=0.76, a=0.1))

w=0.56
b=0.808


We can now view our updated output for $(1, 3)$:

In [31]:
Y = output_with_bias (x=1, w=0.56, b=0.81)
Y

1.37

We also see the loss decrease for $(1, 3)$:

In [32]:
print(squared_difference(y=3, Y=output_with_bias (x=0, w=0, b=0)))
print(squared_difference(y=3, Y=output_with_bias (x=0, w=0, b=0.2)))
print(squared_difference(y=3, Y=output_with_bias (x=0, w=0.56, b=0.76)))
print(squared_difference(y=3, Y=output_with_bias (x=0, w=0.56, b=0.81)))

9
7.839999999999999
5.017600000000001
4.7961


And similarly if we pass $(0, 1)$ through without a subsequent update:

In [33]:
print(squared_difference(y=1, Y=output_with_bias (x=0, w=0, b=0)))
print(squared_difference(y=1, Y=output_with_bias (x=0, w=0, b=0.2)))
print(squared_difference(y=1, Y=output_with_bias (x=0, w=0.56, b=0.76)))
print(squared_difference(y=1, Y=output_with_bias (x=0, w=0.56, b=0.81)))

1
0.6400000000000001
0.0576
0.03609999999999998


So by now, you're probably wondering, "why in the world would we use a neural network for linear regression with one weight and bias and two examples?" Good question.

Let's try a less artificial (but still very artificial) example. Why are we keeping this artificial? Well, if it got any more "real," it'd be hard to examine the mechanisms at work in training the network. We'd be dealing with two many parameters. Am I being too abstract already?

Consider image classification. Neural networks are really good at this. However, if we started with 256 x 256 x 3 images, we'd have too many parameters to visualize what's going on. Similarly, if we let each pixel take values between 0 and 255, it'll be harder to wrap our heads around the math.

So we'll start with another artificial example. Consider a 5x5 "image." This "image" can take pixel values of either 0 or 1. Think of it as a "true" black-and-white image (no grayscale). We can then use this 5x5 image to represent "images" of digits (0 to 9). What would such an image look like? Picture an old-school scoreboard. How about a "0"?

In [34]:
a_zero = [[0,1,1,1,0],[1,0,0,0,1],[1,0,0,0,1],[1,0,0,0,1],[0,1,1,1,0]]
a_zero

[[0, 1, 1, 1, 0],
 [1, 0, 0, 0, 1],
 [1, 0, 0, 0, 1],
 [1, 0, 0, 0, 1],
 [0, 1, 1, 1, 0]]

Or maybe a "4"?

In [35]:
a_four = [[1,0,1,0,0],[1,0,1,0,0],[1,1,1,1,1],[0,0,1,0,0],[0,0,1,0,0]]
a_four

[[1, 0, 1, 0, 0],
 [1, 0, 1, 0, 0],
 [1, 1, 1, 1, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0]]

How about another "4"?

In [36]:
another_four = [[0,0,1,0,0],[0,1,1,0,0],[1,1,1,1,1],[0,0,1,0,0],[0,0,1,0,0]]
another_four

[[0, 0, 1, 0, 0],
 [0, 1, 1, 0, 0],
 [1, 1, 1, 1, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0]]

So now we can envision a neural network that could tell us "that's a four," or "that one's an eight." We call this task **image classification**. But before we go any further, we're going to import our first library. I know, I like doing everything in pure Python, and I'm sure someone reading this is like, "whatever noob, I backprop in assembly." Well whatever right back at you, we're using NumPy. It supports all that matrix stuff that we're gonna need. It's got this thing called a "NumPy array" that's just better for numerical computation, especially with vectors and matrices.

By convention:

In [37]:
import numpy as np

Now let's reencode our "zero" in NumPy:

In [38]:
a_zero_array = np.array([[0,1,1,1,0],[1,0,0,0,1],[1,0,0,0,1],[1,0,0,0,1],[0,1,1,1,0]])
a_zero_array

array([[0, 1, 1, 1, 0],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [0, 1, 1, 1, 0]])

Or we can just pass our `a_zero` list of lists to `np.array`:

In [39]:
also_a_zero_array = np.array(a_zero)
also_a_zero_array

array([[0, 1, 1, 1, 0],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [0, 1, 1, 1, 0]])

We also need a **label** for our array of zeroes, something that will tell us that our array of zeroes is a 0 and not a 1 or a 2. This is analogous to the "true value" of $y$ in our simple regression example.

For classification, we typically use a **one-hot encoding** for each observation. A one-hot encoding is a vector equal in length to the number of classes. In the case of digits, we have 10 classes from 0 to 9. We index the one-hot encoding by these classes, with the "true" class equal to 1 and the others equal to zero. So for a 0, our encoding is (in pure Python and NumPy):

In [40]:
python_zero_encoding = [1,0,0,0,0,0,0,0,0,0]
numpy_zero_encoding = np.array(python_zero_encoding)
print(python_zero_encoding)
print(numpy_zero_encoding)
numpy_zero_encoding

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1 0 0 0 0 0 0 0 0 0]


array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Let's stick with NumPy for here on! (Until we get to PyTorch...)

In [41]:
numpy_four_encoding = np.array([0,0,0,0,1,0,0,0,0,0])
numpy_four_encoding

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

Now let's return to our simple regression example. We can reconceptualize this as a (ridiculously simple) 1x1 matrix mutiplication and addition. Let's take the example of $(0,1)$ with zero-initialized weights and biases. `x` is our 1x1 input, `y` is our 1x1 output, `W` is our 1x1 zero-initialized weight "matrix," and `b` is our bias "vector":

In [42]:
x = np.array([0])
y = np.array([1])
W = np.array([0])
b = np.array([0])
x, y, W, b

(array([0]), array([1]), array([0]), array([0]))

To compute our model output, we do matrix multiplication in NumPy:

In [43]:
Y = np.matmul(W, x) + b
Y

array([0])

We can also compute loss:

In [44]:
se_loss = (y - Y)**2
se_loss

array([1])

We can update our weights too:

In [45]:
def gd_weight_mat (x, y, W, b, a):
    W = W - a * (-2*x*(y - np.matmul(W, x) - b))
    return "W={}".format(W)

What's the "new" weight when `a=0.1`?

In [46]:
print(gd_weight (x, y, W, b, a=0.1))

w=[0.]


Now let's update our bias:

In [47]:
def gd_bias_mat (x, y, W, b, a):
    b = b - a * (-2*(y - W*x - b))
    return "b={}".format(b)

What's the new bias when `b=0.1`?

In [48]:
print(gd_bias (x, y, W, b, 0.1))

b=[0.2]


Good stuff! Now what happens if we apply the same logic to our 5x5 "image" and 10x1 10-class output? Well, we already got our image and output. Now we need to initialize weight and bias matrices. What size do we need? This might seem confusing at first - don't we have a dimentional mismatch between our 5x5 input and 10x1 output? We need to multiply an $n x m$ matrix by an $m x p$ matrix to get an $n x p$ matrix, and we can't even do this with the transpose of our 5 x 5 input...

Ok, the solution is actually simple. We flatten our input matrix into a vector of length 5 x 5 = 25. In NumPy, this is as straightforward as removing the brackets that defined our array as a two-dimensional matrix. Let's use the `shape` attribute to examine our initial array:

In [49]:
a_zero_array = np.array([[0,1,1,1,0],[1,0,0,0,1],[1,0,0,0,1],[1,0,0,0,1],[0,1,1,1,0]])
print(a_zero_array.shape)
print(len(a_zero_array.shape))

(5, 5)
2


Now let's change the shape to a one-dimensional array (vector):

In [50]:
a_zero_vector = np.array([0,1,1,1,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,1,1,1,0])
print(a_zero_vector.shape)
print(len(a_zero_vector.shape))

(25,)
1


One other thing. We actually want the shape of our vector to be (25,1), not (25,). So we can reshape the vector using `reshape`. We could have done this to our 5x5 matrix, but whatever. Also, we're gonna call it `x` since that's what we've been calling our input:

In [51]:
x = a_zero_vector.reshape(25, 1)
x

array([[0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0]])

Now let's define our weights matrix and bias vector. We need something to multiply our (25, 1) vector by to get something of shape (10, 1). Sounds like we need a (10, 25) matrix! Also, we'll have to add a vector to that for bias, and it'll need to be the same shape, so we also need a (10, 1) vector. We'll start with zero initialization, which is stupid but we'll get to that later. `zeros` is a nice built-in function in NumPy that does the trick for us:

In [52]:
W = np.zeros((10, 25))
b = np.zeros((10, 1))

Let's check to make sure that `zeros` did its job for our weight matrix:

In [53]:
W

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

Our bias vector should also be all zeroes (zeros?)

In [54]:
b

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

Cool, now we can compute the output of our model:

In [55]:
output = np.matmul(W, x) + b
output

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

There's one more thing we need to do before computing our loss. We want to convert our model output into a probability distribution to compare it with our ground truth. For this, we use what's called a **softmax** function. The softmax function first exponentiates each of the outputs (they're outputs of the matrix multiplication and addition, but inputs to the softmax function). Then the function divides by the sum of all the exponentiated inputs. In formal terms, the softmax $S(Y)$ applied to the output vector $Y$ is defined as follows:

$S(Y)_{i}$ = $\frac {e^Y_{i}}{\sum_{j=1}^K e^Y_{j}}$ for $i = 1, ..., K$ and $Y = (Y_{i}, ... , Y_{k})$

That might look scary but it's super simple. We use the `exp` function for $e^{Y}$:

In [56]:
def softmax (output):
    return np.exp(output)/np.sum(np.exp(output))

Now we can compute our loss. We'll apply this loss to the output from our softmax function $Y$ and our ground truth $y$. We'll also need to reshape $y$ so that it's the same shape as $Y$:

In [57]:
Y = softmax(output)
y = numpy_zero_encoding.reshape(10,1)

Can anyone guess what `Y` looks like now?

In [58]:
Y

array([[0.1],
       [0.1],
       [0.1],
       [0.1],
       [0.1],
       [0.1],
       [0.1],
       [0.1],
       [0.1],
       [0.1]])

How about `y`?

In [59]:
y

array([[1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

Our loss function is something called **cross-entropy loss**. The cross-entropy function is just the negative natural logarithm of the model output for the "ground truth" class:

$L(y, Y) = - {\sum_{i=1}^K y_{i}log(Y_{i})}$

In other words, we mutiply the ground truth by the natural log (`np.log`) of the model output for each class, sum the results, and flip the sign. Of course, for a one-hot encoding, everything will go to zero except for the ground truth class, so we're just left with the negative natural log of the model output corresponding to the ground truth class. Let's code up a function `cross_entropy` that takes an actual and predicted output and returns the cross-entropy loss:

In [60]:
def cross_entropy(Y, y):
    return -sum(y * np.log(Y))[0]

What's the cross-entropy loss for our current values of `Y` and `y`?

In [61]:
cross_entropy(Y, y)

2.3025850929940455

Alright, now for some more backprop. We need to take the derivative of our (cross-entropy) loss function with respect to the weights and biases. This will require us to apply the chain rule. If you don't remember that from calculus, or haven't taken calculus, don't worry. It's simple. Let $L = L(S)$ be our (cross-entropy) loss (error) function, which takes the output from our softmax function as an input.

Let $S = S(y, Y)$ be our softmax function. $Y = Y(x)$ is the output of our linear transformation $Wx + b$ (for matrix $W$ and vectors $x$ and $b$) or $w_{0}x + b_{0}$ (for a particular weight $w_{0}$ and bias $b_{0}$). For (relative) simplicity, let's differentiate with respect to a particular weight $w_{0}$ and bias $b_{0}$:  

$\partial L/\partial w_{0} = \partial L/\partial S * \partial S/\partial Y * \partial Y/\partial w_{0}$

and

$\partial L/\partial b_{0} = \partial L/\partial S * \partial S/\partial Y * \partial Y/\partial b_{0}$

We'll actually start by computing the derivative of the softmax function $S(y, Y)$ with respect to the output of the linear transformation $Y = w_{0}x + b_{0}$

We'll use the **quotient rule** to compute the derivative:

For $f(x)$ = $\frac{g(x)}{h(x)}$, $f'(x)$ = $\frac{g'(x)h'(x) - h'(x)g(x)}{h(x)^2}$

Recall the softmax function is $S(Y)_{i}$ = $\frac {e^Y_{i}}{\sum_{j=1}^K e^Y_{j}}$

Let's simply things a bit further. Let's only backpropagate one of the weights: the weight connecting the zero-index input with the zero-index output. Recall the input, output, and ground truth:

In [62]:
print(x[0][0])
print(Y[0][0])
print(y[0][0])

0
0.1
1


What does this tell us? We take an input 0 (along with 24 other inputs), pass it through a linear function, apply a softmax transformation, and get 0.1 as our model output. The true output is 1. So what is the softmax function as applied to this specific weight?

When taking the derivative for the same input and output node (the zero index), as we are doing here, we know that $\partial$$e^{a(x)}$/$\partial$$a(x)$ is $e^{a(x)}$. This is just the application of the world's simplest derivative: the derivative of $e^{x}$ is itself.

Now we can apply the quotient rule from above. We'll swap in $Y$ for $x$ since our softmax vector takes the "output" $Y$ as its input:

$g(Y) = e^{a_{i}(Y)}$

$h(Y) = \sum_{k=1}^N e^{a_{k}(Y)}$

$g'(Y) = e^{a_{i}(Y)}$

$h'(Y) = e^{a_{j}(Y)}$ since $e^{a_{k}(Y)} = e^{a_{j}(Y)}$ when $i = j$, and $e^{a_{k}(Y)} = 0$ when $i \ne j$. Remember, we're differentiating the softmax function for $i$, $S_{i}$ with respect to our output $a(Y)$ for $j$, $a_{j}(Y)$

So $f'(Y) = \frac{ e^{a_{i}(Y)} \sum_{j=1}^K e^{a_{j}(Y)} - e^{a_{j}(Y)} e^{a_{i}(Y)} } { (\sum_{k=1}^N e^{a_{k}(Y)})^2 }$

Factoring out an $e^{a_{i}(Y)}$ from the numerator:

$f'(Y) = \frac{ e^{a_{i}(Y)} (\sum_{j=1}^K e^{a_{j}(Y)} - e^{a_{j}(Y)}) } { (\sum_{j=1}^K e^{a_{k}(Y)})^2 }$

Splitting the expression:

$f'(Y) = \frac{ e^{a_{j}(Y)} } { \sum_{j=1}^K e^{a_{k}(Y)} } Y \frac{ \sum_{k=1}^N e^{a_{k}(Y)} - e^{a_{j}(Y)} } { \sum_{k=1}^N e^{a_{k}(Y)} }$

... which is magically equivalent to $S_{i}(1 - S_{j})$ (again, for our case where $i=j$)

Now let's return to our cross-entropy loss:

$L = - \sum_{i=1}^N y_{i} ln(S_{i})$, where $y$ is our "ground truth" vector, and thus $y_{i}$ is index $i$ of the "ground truth" vector. Again, $S_{i}$ is the softmax function applied to index $i$ of the "model" or output vector.

If we differentiate our cross-entropy loss with respect to our output for index $i$, we get:

$\partial L/ \partial Y_{i} = - \sum_{k=1}^N y_{k} \frac{ \partial{ln(S_{k})} } { \partial{ Y_{i} }}$

Applying the chain rule:

$\partial L/ \partial Y_{i} = - \sum_{k=1}^N y_{k} \frac{ \partial{ln(S_{k})} } { \partial{ S_{k} }} x \frac{ \partial{S_{k}} }{ \partial{Y_{i}} }$

Since $\frac {\partial ln(x) }{\partial{x}} = \frac{1}{x}$,

$\partial L/ \partial Y_{i} = - \sum_{k=1}^N y_{k} \frac{1}{S_{k}} x \frac{ \partial{S_{k}} }{ \partial{Y_{i}} }$

Now let's plug in our softmax derivative. We can decompose the expression into instances where $k=i$ and $k \ne i$:

$\partial L/ \partial Y_{i} = - y_{i}(1 - S_{i}) - \sum_{k \ne i}^N y_{k} \frac{1}{S_{k}}(-S_{k} S_{i})$

$= - y_{i}(1 - S_{i}) + \sum_{k \ne i}^N y_{k} S_{i}$

$= - y_{i} + y_{i} S_{i} + \sum_{k \ne i}^N y_{k} S_{i}$

$= S_{i} (y_{i} + \sum_{k \ne i}^N y_{k}) - y_{i}$

Since $y$ sums up to 1 (it's one 1 and a bunch of zeroes - a "one-hot" encoding), $\sum_{k = 1}^N y_{k} = 1$ and $y_{i} + \sum_{k \ne 1}^N y_{k} = 1$

Thus, $\frac {\partial L}{\partial Y_{i}} = S_{i} - y_{i}$

Yikes, that's a lot of math. The code is ridiculously simpler:

In [63]:
def grad_cross_entropy(Y, y):
    return softmax(Y) - y

Let's call `grad_cross_entropy` on our current `Y` and `y`:

In [64]:
grad_cross_entropy(Y, y)

array([[-0.9],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1],
       [ 0.1]])

We need to take a step further and update our weights $w_{i}$. Applying the chain rule:

$\frac{\partial L}{\partial w_{i}} = \frac{\partial L}{\partial Y_{i}} x \frac{\partial Y_{i}}{\partial w_{i}}$

$= (S_{i} - y_{i}) x_{i}$

We can now update our weights!

$w_{i(t+1)} = w_{it} - \alpha \frac{\partial Y}{\partial w_{it}}$

$= w_{it} - \alpha \frac{\partial Y}{\partial Y_{i}} \frac{\partial Y_{i}}{\partial w_{i}}$

Let's first update the single weight $w_{0}$. Can we write a function that does this?

In [65]:
def gd_cross_entropy (Y, y, x, w, a):
    return w - a*(softmax(Y[0]) - y[0])*x

Recall that $x=0$. We'll keep our learning rate $\alpha=0.1$. What does this function return?

In [66]:
gd_cross_entropy(Y=Y, y=y, x=0, w=0, a=0.1)

array([0.])

Uh oh, looks like our weights aren't going to update again with $x = 0$. 

We could try with a different observation or different weight, but at this point we should note that zero initialization isn't ideal. Once we start adding activation functions, zero initialization will prevent our neural network from updating weights during backpropagation. More on this once we get to activation functions.

Let's try something better: randomly initialized weights (and biases)! Recall that we have a 10x25 weight matrix `W` and a 10x1 bias vector `b`. We're going to replace those zero initializations with weights randomly sampled from a standard normal distribution (mean 0, variance 1). There's a NumPy function that does this called `numpy.random.randn`:

In [67]:
W = np.random.randn(10, 25)
b = np.random.randn(10, 1)

Let's make sure our weight matrix passes the eyeball test:

In [68]:
W

array([[-1.05484385,  0.81367903,  0.57850297, -1.53256912,  0.04445959,
        -0.1148211 ,  0.92687145, -0.47512158, -0.69612471,  0.2249142 ,
        -1.94512833, -0.24495539, -1.32213242,  1.41051532,  1.28525731,
        -0.95848589, -0.3126557 , -0.49553449,  0.43346275,  0.55650262,
         0.67802816, -0.76460229,  0.80934738,  1.08350917,  0.65523911],
       [ 1.80128161, -0.28263696,  0.623465  ,  1.43597445,  0.31488569,
         0.03171404, -0.97351093,  0.46709336, -0.70068404,  1.82473893,
        -0.98601865,  0.03645138, -0.71914953, -0.62288268, -0.22857658,
        -0.77976512, -1.59740654,  1.26060775,  1.83354586,  0.19612898,
        -0.49823872, -0.7696657 ,  0.65299849,  0.81190947,  0.87304711],
       [-0.80924751, -1.87790484, -0.13191904,  0.68639244,  1.10371921,
         0.23672855,  0.40186601, -1.02306624, -0.67332883, -0.24744444,
        -1.26677578, -0.39753989, -0.77366075, -1.72926211,  1.61687452,
        -0.12992479,  0.48237176, -1.11008633, -0

Our bias vector is more pleasing to the eyeballs:

In [69]:
b

array([[ 1.20271764],
       [ 2.08457018],
       [ 0.5503911 ],
       [-0.30176484],
       [-0.38678701],
       [-0.31446401],
       [ 1.10029442],
       [-1.05853278],
       [ 0.09882997],
       [-0.35766539]])

Let's compute our outputs again with these randomly initialized weights:

In [70]:
Y = np.matmul(W, x) + b
Y

array([[ 1.23882361],
       [ 4.61483655],
       [ 1.99479822],
       [-3.32811171],
       [ 3.58462725],
       [-2.33802756],
       [ 7.18746098],
       [ 0.54074097],
       [-0.31431916],
       [ 3.89948129]])

Now let's pass that output through a softmax function:

In [71]:
S = softmax(Y)
S

array([[2.26701235e-03],
       [6.63189554e-02],
       [4.82802477e-03],
       [2.35536967e-05],
       [2.36713739e-02],
       [6.33938547e-05],
       [8.68788915e-01],
       [1.12792559e-03],
       [4.79658950e-04],
       [3.24311864e-02]])

...and compute cross entropy loss:

In [72]:
cross_entropy(S, y)

6.089292459946072

Now backprop is going to get trickier. We have to update a bunch of weights and biases, and pretty soon we're going to be stacking layers. Hopefully by now you have a good idea of what we're doing -- it's just the partial derivative of the loss function with respect to the weights and biases, and we solve for this using the chain rule. Maybe in a future course, we'll do some more backprop in NumPy, but I think this is sufficient for an introduction to neural networks. We got a lot more to cover.

Thankfully, we have something called **PyTorch** that makes things a lot easier. PyTorch is a **deep learning framework**, or collection of operations for training neural networks. Also, it looks a lot like NumPy. You'll have to have it [installed](https://pytorch.org/get-started/locally/) already. Let's import it:

In [73]:
import torch

Just as the basic unit of NumPy is the numpy array, PyTorch has a class called `Tensor`. We'll be performing our operations on `Tensor`s. PyTorch makes it easy to convert NumPy arrays to Tensors using `torch.tensor`, so we're gonna do that for our `W` matrix, and `b`, `x`, and `y` vectors.

Before we get to this, we'll also change our encoding of the **target** (ground truth) from a one-hot vector to class indices. We simply do this by passing the label value, in this case `0` to `torch.tensor` and setting the output to `y`:

In [74]:
y = torch.tensor([0])

Let's confirm that our label is in fact a `torch.tensor`. One simple way to do this is to type `y` in the interactive cell and execute the cell:

In [75]:
y

tensor([0])

We could also call `type` on `y`:

In [76]:
type(y)

torch.Tensor

Note that this is different than calling `type` as a `Tensor` method:

In [77]:
y.type()

'torch.LongTensor'

If we called `type` without the parentheses, we'd get something different:

In [78]:
y.type

<function Tensor.type>

Not to mention we could also call the `dtype` method:

In [79]:
y.dtype

torch.int64

Ok, we're having too much fun. While we're at it, what's the `type` of our bias vector?

In [80]:
type(b)

numpy.ndarray

Now let's convert `W`, `b`, `x`, and `y` from `numpy.ndarray`s to `torch.Tensor`s. We're also going to pass another parameter to enable computation of gradients, `requires_grad=True`. Note that we don't want to do this for our data -- just our weights and biases. We set `requires_grad=False` for those instead. We'll also need to convert our input datum `x` to a float using the `float` method (to do math and stuff). Let's also make sure to do this for `y` to ensure that gradient calculation is disabled:

In [81]:
W = torch.tensor(W, requires_grad=True)
b = torch.tensor(b, requires_grad=True)
x = torch.tensor(x, requires_grad=False).float()
y = torch.tensor(y, requires_grad=False)

/Users/pp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


What does our weight matrix look like now?

In [82]:
W

tensor([[-1.0548,  0.8137,  0.5785, -1.5326,  0.0445, -0.1148,  0.9269, -0.4751,
         -0.6961,  0.2249, -1.9451, -0.2450, -1.3221,  1.4105,  1.2853, -0.9585,
         -0.3127, -0.4955,  0.4335,  0.5565,  0.6780, -0.7646,  0.8093,  1.0835,
          0.6552],
        [ 1.8013, -0.2826,  0.6235,  1.4360,  0.3149,  0.0317, -0.9735,  0.4671,
         -0.7007,  1.8247, -0.9860,  0.0365, -0.7191, -0.6229, -0.2286, -0.7798,
         -1.5974,  1.2606,  1.8335,  0.1961, -0.4982, -0.7697,  0.6530,  0.8119,
          0.8730],
        [-0.8092, -1.8779, -0.1319,  0.6864,  1.1037,  0.2367,  0.4019, -1.0231,
         -0.6733, -0.2474, -1.2668, -0.3975, -0.7737, -1.7293,  1.6169, -0.1299,
          0.4824, -1.1101, -0.9505, -0.4738, -0.4384,  1.3970,  0.2512,  1.3840,
         -0.6754],
        [ 0.1114, -0.9876, -0.6144, -0.6816,  0.7047, -1.7680, -1.6470,  1.1304,
          1.1096,  0.3625,  0.3828, -1.5768,  2.7148,  1.1167,  0.7278, -0.6968,
          0.0269,  1.5446,  0.1131, -0.0444, -0.1955

How about our bias vector?

In [83]:
b

tensor([[ 1.2027],
        [ 2.0846],
        [ 0.5504],
        [-0.3018],
        [-0.3868],
        [-0.3145],
        [ 1.1003],
        [-1.0585],
        [ 0.0988],
        [-0.3577]], dtype=torch.float64, requires_grad=True)

Let's confirm that our input is also a `tensor`:

In [84]:
x

tensor([[0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.]])

Finally, let's examine our output label:

In [85]:
y

tensor([0])

Note that we haven't yet computed gradients:

In [86]:
print(W.grad)
print(b.grad)

None
None


Now let's do our linear transformation (matrix multiplication and addition):

In [87]:
Y = torch.mm(W, x.double())
Y = Y.double()
Y = Y.add(b)
Y

tensor([[ 1.2388],
        [ 4.6148],
        [ 1.9948],
        [-3.3281],
        [ 3.5846],
        [-2.3380],
        [ 7.1875],
        [ 0.5407],
        [-0.3143],
        [ 3.8995]], dtype=torch.float64, grad_fn=<AddBackward0>)

Let's reshape our output from a column vector to a row vector. It'll become clear soon why we do this:

In [88]:
print(Y.shape)
Y = Y.view(1, -1)
print(Y.shape)
Y

torch.Size([10, 1])
torch.Size([1, 10])


tensor([[ 1.2388,  4.6148,  1.9948, -3.3281,  3.5846, -2.3380,  7.1875,  0.5407,
         -0.3143,  3.8995]], dtype=torch.float64, grad_fn=<ViewBackward>)

PyTorch has a built-in `softmax` function. We import it from the `torch.nn.functional` module:

In [89]:
import torch.nn.functional as F

What's the output of our softmax function?

In [90]:
S = F.softmax(Y)
S

/Users/pp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[2.2670e-03, 6.6319e-02, 4.8280e-03, 2.3554e-05, 2.3671e-02, 6.3394e-05,
         8.6879e-01, 1.1279e-03, 4.7966e-04, 3.2431e-02]], dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)

Can we confirm that our `softmax` output sums to 1?

In [91]:
sum(S)

tensor([2.2670e-03, 6.6319e-02, 4.8280e-03, 2.3554e-05, 2.3671e-02, 6.3394e-05,
        8.6879e-01, 1.1279e-03, 4.7966e-04, 3.2431e-02], dtype=torch.float64,
       grad_fn=<AddBackward0>)

Whoops! One more thing:

In [92]:
sum(sum(S))

tensor(1., dtype=torch.float64, grad_fn=<AddBackward0>)

When computing cross-entropy loss in PyTorch, we just use the raw output -- it's not necessary to pass the output through a softmax function first:

In [93]:
E = F.cross_entropy(Y.view(1, -1), y)
E

tensor(6.0893, dtype=torch.float64, grad_fn=<NllLossBackward>)

Backpropagation is easy in PyTorch! Just use `.backward()`. We can do this to our linear transformation:

In [94]:
E.backward()

Now let's check out our gradients for our weight matrix!

In [95]:
print(W.grad)

tensor([[-0.0000e+00, -9.9773e-01, -9.9773e-01, -9.9773e-01, -0.0000e+00,
         -9.9773e-01, -0.0000e+00, -0.0000e+00, -0.0000e+00, -9.9773e-01,
         -9.9773e-01, -0.0000e+00, -0.0000e+00, -0.0000e+00, -9.9773e-01,
         -9.9773e-01, -0.0000e+00, -0.0000e+00, -0.0000e+00, -9.9773e-01,
         -0.0000e+00, -9.9773e-01, -9.9773e-01, -9.9773e-01, -0.0000e+00],
        [ 0.0000e+00,  6.6319e-02,  6.6319e-02,  6.6319e-02,  0.0000e+00,
          6.6319e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  6.6319e-02,
          6.6319e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  6.6319e-02,
          6.6319e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  6.6319e-02,
          0.0000e+00,  6.6319e-02,  6.6319e-02,  6.6319e-02,  0.0000e+00],
        [ 0.0000e+00,  4.8280e-03,  4.8280e-03,  4.8280e-03,  0.0000e+00,
          4.8280e-03,  0.0000e+00,  0.0000e+00,  0.0000e+00,  4.8280e-03,
          4.8280e-03,  0.0000e+00,  0.0000e+00,  0.0000e+00,  4.8280e-03,
          4.8280e-03,  0.0000e+00,  

How about the bias vector?

In [96]:
print(b.grad)

tensor([[-9.9773e-01],
        [ 6.6319e-02],
        [ 4.8280e-03],
        [ 2.3554e-05],
        [ 2.3671e-02],
        [ 6.3394e-05],
        [ 8.6879e-01],
        [ 1.1279e-03],
        [ 4.7966e-04],
        [ 3.2431e-02]], dtype=torch.float64)


Isn't this magical? But wait, there's more. We don't even need to define our weights and biases manually. PyTorch gives us (at least) two options: the `torch.nn` module and its `torch.nn.Functional` API. We've already imported `nn.Functional` and used it for computing softmax and cross-entropy loss. Let's now import `torch.nn`:

In [97]:
import torch.nn as nn

We can use the `nn.Linear` function to define our weight matrix and bias vector. We just need to pass our number of input nodes (`in_features`) and output nodes (`out_features`), and specify `bias=True` to use biases. This is called a **fully-connected layer** since all of the input nodes are connected to all of the output nodes. Later, we'll encounter another architecture in which weights are shared and each input node is not connected to each output node. By convention, we call our fully-connected layer `fc`:

In [98]:
fc = nn.Linear(25, 10)
print(fc)

Linear(in_features=25, out_features=10, bias=True)


We can view our weights with the `weight` method:

In [99]:
fc.weight

Parameter containing:
tensor([[-0.1352,  0.1718, -0.1566, -0.1154,  0.1245, -0.0324,  0.0596, -0.0447,
         -0.1963,  0.1383,  0.1284, -0.1122,  0.1974,  0.0162,  0.1554,  0.0871,
          0.0487,  0.0691,  0.1488,  0.0367,  0.0262,  0.1898, -0.1040, -0.0492,
         -0.1991],
        [-0.1212,  0.0909,  0.0600, -0.0566,  0.0442,  0.0538, -0.1232, -0.1775,
         -0.1838,  0.1921, -0.0327, -0.0326, -0.1011, -0.0048,  0.0761,  0.0188,
          0.1134, -0.0076, -0.0581,  0.1999, -0.1977,  0.0572, -0.1367, -0.0361,
         -0.0736],
        [-0.0017,  0.0714,  0.0404, -0.1737,  0.1339,  0.0747,  0.1863, -0.0490,
         -0.0573, -0.1115, -0.1305, -0.0613, -0.1434,  0.0458, -0.0811,  0.1144,
          0.1894, -0.0490, -0.0884, -0.1162, -0.0207,  0.0305,  0.0791, -0.1320,
         -0.0612],
        [-0.0654,  0.0730,  0.1376,  0.0264, -0.1992, -0.0625, -0.1488,  0.1655,
         -0.1142, -0.0747, -0.1564, -0.0325,  0.0932,  0.0353,  0.1189,  0.1244,
          0.0367, -0.1155, -0.

We can also view our biases with `bias`:

In [100]:
fc.bias

Parameter containing:
tensor([ 0.1001, -0.0095,  0.1320,  0.0117,  0.0808,  0.1041,  0.0070,  0.0589,
        -0.1526,  0.0888], requires_grad=True)

Cool, now let's construct our network the way it's supposed to be constructed. We define a class `Net` that inherits the properties of `nn.Module`, the "base class for all neural network modules" ([line 32](https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/module.py)) When we build a neural network, we construct a subclass of this class. We start by defining the class and our `__init__` constructor, then add our `fc` layer as a method of `self`:

In [101]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(25, 10)

We also need to define how our input will **forward propagate** through the network. By convention, we define another class method `forward` that takes our randomly-initialized weights and biases from self and an input vector (or matrix of concatenated input vectors, but we'll get there eventually!) and produces our network output (from which we can then apply cross-entropy loss and update our weights and biases). We just need to pass our input through our fc layer and return the output (we'll add other stuff later):

In [102]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(25, 10)
    def forward(self, x):
        return self.fc(x)

We can view our network architecture by calling `net` as an instance of `Net()`:

In [103]:
net = Net()
net

Net(
  (fc): Linear(in_features=25, out_features=10, bias=True)
)

Not much to see here! Don't worry, we're just getting started. Think of `Net` as an **architecture** that generates randomly-initialized untrained models (here, `net`). We then use instances such as net to train neural networks on data. We can now pass our input `x` through the `net` instance of `Net()` to generate a predicted model output `Y`. 

However, we first need to do two things to our input tensor `x`: change it from a (25, 1) column vector to a (1, 25) row vector using `view(1, -1)`, and cast it from a double to a float using `.float()`:

In [104]:
x = x.view(1, -1)
x = x.float()

Now we're good to go! What's our model output look like?

In [105]:
Y = net(x)
Y

tensor([[-0.5602,  0.3292,  0.0777, -0.0061, -0.6894, -0.3403,  0.3445,  0.2476,
          0.1991, -0.4787]], grad_fn=<AddmmBackward>)

We can now apply cross-entropy loss to our model output `Y` and label `y`. Let's set the output to the variable `E` and call it: 

In [106]:
E = F.cross_entropy(Y, y)
E

tensor(2.8407, grad_fn=<NllLossBackward>)

And finally, let's update our weights and biases:

In [107]:
E.backward()

We can loop over the **parameters** method of our **net** instance and print **param.data** to view our updated weights and biases:

In [108]:
for param in net.parameters(): print(param.data)

tensor([[ 1.8709e-01,  1.7630e-02, -1.3987e-01, -1.1207e-01,  8.4075e-02,
          1.0431e-01,  1.6378e-01, -1.8669e-01, -1.2836e-01, -1.6410e-01,
          1.7724e-01, -1.2798e-01, -1.0449e-02, -7.6011e-03,  1.2943e-02,
         -8.3517e-02, -1.6593e-01,  1.9548e-01, -1.4061e-02, -1.8278e-01,
         -1.4145e-02, -1.9412e-01,  1.0410e-01,  2.4284e-02, -1.1030e-01],
        [-8.9069e-03,  8.7440e-02,  3.8122e-03,  1.7244e-01, -2.5063e-02,
          2.4075e-02, -4.4239e-03, -1.3070e-01,  7.4625e-02, -1.5466e-01,
          1.6669e-01, -7.2566e-02, -1.1074e-01,  9.9252e-02,  1.2892e-01,
          8.2317e-02,  6.6966e-02, -1.3729e-01,  6.3897e-02, -8.9696e-02,
         -6.1463e-02, -5.7353e-02, -2.6082e-02, -1.8872e-01, -7.0256e-02],
        [ 8.0930e-03, -1.3951e-01,  1.2391e-01,  8.0808e-02,  4.2698e-03,
          1.9735e-01,  1.2157e-01,  6.7829e-02,  1.0580e-01, -2.1023e-02,
         -4.7460e-03,  5.1834e-02,  9.1520e-03,  1.0622e-01,  1.9248e-01,
         -1.6679e-01,  1.2071e-01,  

Wow, we've done a lot already, and we haven't even passed more than TWO points of data through our (absurdly simple) network. However, before we move on to lots of observations with real image data(!!!!) we're going to introduce a few more operations that we'll perform on our data to obtain more accurate classification results with greater computational efficiency.

Let's think about how we're approaching the problem of image classification. Right now we're assuming that the way to go is to treat each pixel as an input, and to learn each connection between pixel and output class. Since we're dealing with a ridiculously small "image" (25 1-bit pixels), we don't have to learn too many parameters: 10x25 weights + 10 biases = 250 + 10 = 260 parameters.

Now if we're dealing with a "standard" RGB image, we have to ingest a ton more pixels: 256 x 256 x 3 = 196,608 8-bit pixels in our input layer, each taking a value between 0 and 255 (and this is a "small" datum compared to a video or a genome, among other data). Even if we just apply a single matrix multiplication and addition to transform our data from a vector of length 196,608 to a vector of length 10, we still need to learn 196,608 x 10 weights + 10 biases = 1,966,090 parameters. That's a lot of parameters for a single linear transformation! Especially since it's completely insufficient for image classification. No wonder no one used neural networks for so long...

This brings us to our second issue: a "vanilla" matrix multiplication and addition isn't the best way to extract information from an image, whether it be for the purposes of classification, object detection (drawing bounding boxes around objects of interest in an image), or semantic segmentation (coloring the pixels of objects of interest in an image).

It turns out we can solve both of these issues with an operation called a **convolution**. We'll use our sample 5x5 "zero array" of one-bit pixels to demonstrate how this works:

In [109]:
a_zero_array

array([[0, 1, 1, 1, 0],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [0, 1, 1, 1, 0]])

Recall that we want to learn a set of weights and biases that'll connect our input pixels with our output. The output is a one-hot encoded vector corresponding to the "zero" label. Recall that we already constructed this array in NumPy:

In [110]:
numpy_zero_encoding

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

We learned our weights and biases (respectively) from a randomly-initialized 10x25 matrix and 10x1 vector. We called our weight matrix `W`:

In [111]:
W

tensor([[-1.0548,  0.8137,  0.5785, -1.5326,  0.0445, -0.1148,  0.9269, -0.4751,
         -0.6961,  0.2249, -1.9451, -0.2450, -1.3221,  1.4105,  1.2853, -0.9585,
         -0.3127, -0.4955,  0.4335,  0.5565,  0.6780, -0.7646,  0.8093,  1.0835,
          0.6552],
        [ 1.8013, -0.2826,  0.6235,  1.4360,  0.3149,  0.0317, -0.9735,  0.4671,
         -0.7007,  1.8247, -0.9860,  0.0365, -0.7191, -0.6229, -0.2286, -0.7798,
         -1.5974,  1.2606,  1.8335,  0.1961, -0.4982, -0.7697,  0.6530,  0.8119,
          0.8730],
        [-0.8092, -1.8779, -0.1319,  0.6864,  1.1037,  0.2367,  0.4019, -1.0231,
         -0.6733, -0.2474, -1.2668, -0.3975, -0.7737, -1.7293,  1.6169, -0.1299,
          0.4824, -1.1101, -0.9505, -0.4738, -0.4384,  1.3970,  0.2512,  1.3840,
         -0.6754],
        [ 0.1114, -0.9876, -0.6144, -0.6816,  0.7047, -1.7680, -1.6470,  1.1304,
          1.1096,  0.3625,  0.3828, -1.5768,  2.7148,  1.1167,  0.7278, -0.6968,
          0.0269,  1.5446,  0.1131, -0.0444, -0.1955

`b` is our bias vector:

In [112]:
b

tensor([[ 1.2027],
        [ 2.0846],
        [ 0.5504],
        [-0.3018],
        [-0.3868],
        [-0.3145],
        [ 1.1003],
        [-1.0585],
        [ 0.0988],
        [-0.3577]], dtype=torch.float64, requires_grad=True)

The assumption here is that each class is best learned (in terms of accuracy and computational efficiency) by summing up weighted values associated with each pixel for that class. Is this really optimal? An alternative is to use a **convolutional neural network** that learns sets of shared weights called **filters**, **kernels**, or **receptive fields** (do we really need four names for the same thing?) These shared weights take a particular **shape** (typically square or more generally rectangular). Let's start with the example of the smallest possible (1x1) filter. First we need to decide how many filters to learn. We call this our output or **volume**. Let's stick with volume since output can refer to other things. What if we only learned one filter? Let's randomly initialize our filter. We'll initialize it as an array even though it's just a single value. We'll stick with `w` to denote our weight:

In [113]:
w = np.random.randn(1)
w

array([-0.39092161])

Now we need to **stride** the filter across our "image." At each point, we take the dot product of our (1x1) filter and the corresponding array of pixels of the same size (also 1x1). Let's start with a stride of 1! Our single volume $V$ is the (25x1) output of a for loop over the pixel vector. Let's define a function `conv_1x1_stride1` that takes an input pixel vector `X`, randomly initializes a 1x1 filter, strides the filter one pixel at a time across the image, and returns the output:

In [114]:
def conv_1x1_stride1(X):
    w = np.random.randn(1)
    return X[0:len(X)]*w

Now let's pass our specific input vector `a_zero_vector` to the function and store the result as `V` (for **volume**):

In [115]:
V = conv_1x1_stride1(X=a_zero_vector)
V

array([0.       , 0.3911008, 0.3911008, 0.3911008, 0.       , 0.3911008,
       0.       , 0.       , 0.       , 0.3911008, 0.3911008, 0.       ,
       0.       , 0.       , 0.3911008, 0.3911008, 0.       , 0.       ,
       0.       , 0.3911008, 0.       , 0.3911008, 0.3911008, 0.3911008,
       0.       ])

How would we change the function for a stride of 2? We can just add a second colon and "2" to tell Python (NumPy?) to select every other entry beginning with index 0 (0, 2, 4, etc.) Let's call this function `conv_1x1_stride2`:

In [116]:
def conv_1x1_stride2(X):
    w = np.random.randn(1)
    return X[0:len(X):2]*w

What happens if we apply this convolution to our vector of zeros?

In [117]:
conv_1x1_stride2(X=a_zero_vector)

array([0.        , 0.89818267, 0.        , 0.        , 0.        ,
       0.89818267, 0.        , 0.89818267, 0.        , 0.        ,
       0.        , 0.89818267, 0.        ])

Generalizing to a stride `s` is as simple as indexing the array by every `s`th entry. Of course we can't stride the kernel by zero or a negative value. What would this function look like? Let's call it `conv_1x1`:

In [118]:
def conv_1x1(X, s):
    if s > 0:
        w = np.random.randn(1)
        return X[0:len(X):s]*w
    return "you can't do that"

What happens when we pass our image through `conv_1x1` with a stride of 5?

In [119]:
conv_1x1(X=a_zero_vector, s=5)

array([-0.        , -1.24676061, -1.24676061, -1.24676061, -0.        ])

We can also learn multiple volumes by generating a two-dimensional weight matrix `W`. The first dimension is the kernel size (in this case just 1), and the second dimension is our number of output volumes `k`, which we loop over to generate multiple outputs. How would we code this? I stored the result as a list of arrays; don't know if that's kosher.

In [120]:
def conv_1x1(X, s, k):
    V = []
    if s > 0:
        W = np.random.randn(1,k)
        [V.append(X[0:len(X):s]*W[0][i]) for i in range(k)]
        return V
    return "you can't do that"

What happens if we apply this 1x1 convolution to our zero vector with a stride of 5 and four output volumes?

In [121]:
conv_1x1(X=a_zero_vector, s=5, k=4)

[array([-0.        , -0.58719386, -0.58719386, -0.58719386, -0.        ]),
 array([-0.        , -0.86607815, -0.86607815, -0.86607815, -0.        ]),
 array([0.        , 0.20199174, 0.20199174, 0.20199174, 0.        ]),
 array([-0.        , -0.92015104, -0.92015104, -0.92015104, -0.        ])]

How might we define a 2x2 convolution? We first need to initialize a filter with random values. Again, let's assign our output to the weight matrix `W` and call it:

In [122]:
W = np.random.randn(2,2)
W

array([[-0.4036377 , -2.63969468],
       [-0.54287506, -1.11070243]])

Now we need to define the region over which we'll stride. Let's view the "image" as a 5x5 array:

In [123]:
X = a_zero_array
X

array([[0, 1, 1, 1, 0],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [0, 1, 1, 1, 0]])

What's the value of the first row and second column of this array?

In [124]:
a_zero_array[0][1]

1

The convolution is first applied to indices (0,0),(0,1),(1,0),(1,1). We then stride horizontally by $s$ steps. Let's start with a stride of one $(s=1)$ and a volume of one. We'll add 1 to each $y$, yielding indices (0,1),(0,2),(1,1),(1,2).

Note that $x$ is the vertical index (from the top) and $y$ is the horizontal index (also from the top). This is kinda weird from a Cartesian perspective but makes sense in terms of how we define NumPy arrays.

What are the dimensions of our volume matrix? 

Our width, $W_{2}$ is $(W_{1}-F+2P)/S) + 1$

Our height, $H_{2}$ is $(H_{1}-F+2P)/S) + 1$

and our depth, $D_{2}$ is $K$

where $W_{1}$ and $H_{1}$ are the respective width and height of the input "image." In this particular case, $W_{1} = H_{1} = 5$

$F$ is the width and height of our filter(s). Here we assume that our filter(s) are square. $S$ is our stride. For our example, $F=2$ and $S=1$

$D$ is the number of filters in our output volume, $V$. This is explicitly defined as a parameter (we're not going to do this in our first example (where $D=1$) but we will need to account for this in our generic convolution function).

$P$ is something called **zero padding**. This adds $P$ layers of zeros to the border of our input image. Padding is useful when we want to use all the information provided by the image but the image and kernel size don't neatly align. For instance, we might want to use a 3x3 kernel on our 5x5 image with a stride of 3, but we would be unable to compute the dot product on the fourth and fifth rows or columns without striding across the border of the image. However, with one layer of zero padding we can now compute the first dot product on a row and column of zeros, plus the entries corresponding to the first and second row and column. Our second dot product will now use the top row of padding in addition to the intersection of the top two rows and rightmost three columns. We then use the zero padding for the first column of our third dot product. The fourth dot product is computed on the bottom 3x3 of the image (without padding). We're not going to use padding in our first example, but we want to make sure to include it when we build our more general function.

Source: http://cs231n.github.io/convolutional-networks/

So for our volume matrix of size $(W_{2},H_{2})$ (an instance of a volume tensor of size $(W_{2},H_{2},K_{2})$),

$W_{2} = (W_{1}-F+2P)/S) + 1 = (5 - 2 + 2(0))/1 + 1 = 3/1 + 1 = 4$

$H_{2} = (H_{1}-F+2P)/S) + 1 = (5 - 2 + 2(0))/1 + 1 = 3/1 + 1 = 4$

Let's do our first convolution. We can do this by indexing the top 2x2 slice of X and multiplying by W. Interestingly, this produces the elementwise product of two matrices, whereas `numpy.dot` is equivalent to `numpy.matmul` for matrices. Weird, huh. What's the output of this operation?

In [125]:
W*X[0:2,0:2]

array([[-0.        , -2.63969468],
       [-0.54287506, -0.        ]])

What happens if we sum up over each of the columns?

In [126]:
sum(W*X[0:2,0:2])

array([-0.54287506, -2.63969468])

If we do another `sum`, we get the desired output of our convolution. Sum of elementwise product, dot product, or both? I'm not sure. I got my PhD in political science. What's the output?

In [127]:
sum(sum(W*X[0:2,0:2]))

-3.1825697374985795

Now we need to stride this thing. What's our next slice of the "image"? We just add one to the "second" dimension of our slice:

In [128]:
X[0:2,0+1:2+1]

array([[1, 1],
       [0, 0]])

We'll continue to do this until we stride horizontally across the entire array. Then we'll stride vertically until we do the same in both dimensions. We can start again with a blank output list $V$. As we stride, we append our "dot product" (or whatever it is) to $V$:

In [129]:
V=[]
V.append(sum(sum(W*X[0:2,0:2])))
print(V)
V.append(sum(sum(W*X[0:2,0+1:2+1])))
print(V)
V.append(sum(sum(W*X[0:2,0+1+1:2+1+1])))
print(V)

[-3.1825697374985795]
[-3.1825697374985795, -3.0433323777571775]
[-3.1825697374985795, -3.0433323777571775, -3.0433323777571775]


See what's going on?

In [130]:
V=[]
m=0
n=2
V.append(sum(sum(W*X[0:2,m:n])))
print(V)
m+=1
n+=1
V.append(sum(sum(W*X[0:2,m:n])))
print(V)
m+=1
n+=1
V.append(sum(sum(W*X[0:2,m:n])))
print(V)

[-3.1825697374985795]
[-3.1825697374985795, -3.0433323777571775]
[-3.1825697374985795, -3.0433323777571775, -3.0433323777571775]


Let's turn our first horizontal stride into a for loop:

In [131]:
V=[]
m=0
n=2
for i in range(5):
    V.append(sum(sum(W*X[0:2,m:n])))
    m+=1
    n+=1
print(V)

[-3.1825697374985795, -3.0433323777571775, -3.0433323777571775, -1.514340124012934, -1.653577483754336]


Good stuff. Now we can stride vertically:

In [132]:
V=[]
V.append(sum(sum(W*X[1:3,0:2])))
print(V)
V.append(sum(sum(W*X[1:3,1:3])))
print(V)
V.append(sum(sum(W*X[1:3,2:4])))
print(V)

[-0.946512753069815]
[-0.946512753069815, 0.0]
[-0.946512753069815, 0.0, 0.0]


And so on... We're doing the same operation over again, this time starting with 1:3 instead of 0:2. Another loop?

In [133]:
V=[]
h=0
k=2
m=0
n=2
for i in range(5):
    for j in range(5):
        V.append(sum(sum(W*X[h:k,m:n])))
        m+=1
        n+=1
    h+=1
    k+=1
    m=0
    n=2
V=np.asarray(V)
V=V.reshape(5,5)
print(V)

[[-3.18256974 -3.04333238 -3.04333238 -1.51434012 -1.65357748]
 [-0.94651275  0.          0.         -3.75039711 -4.69690986]
 [-0.94651275  0.          0.         -3.75039711 -4.69690986]
 [-1.51434012 -1.65357748 -1.65357748 -3.18256974 -3.04333238]
 [-3.75039711 -4.69690986 -4.69690986 -0.94651275  0.        ]]


Sweet! How would we turn this into a function? Let's call it `conv_2x2`:

In [134]:
def conv_2x2(X):
    W=np.random.randn(2,2)
    V=[]
    h=0
    k=2
    m=0
    n=2
    for i in range(5):
        for j in range(5):
            V.append(sum(sum(W*X[h:k,m:n])))
            m+=1
            n+=1
        h+=1
        k+=1
        m=0
        n=2
    V=np.asarray(V)
    V=V.reshape(5,5)
    return V

What happens when we pass our "image" through the function?

In [135]:
conv_2x2(X=X)

array([[-1.4400216 , -0.65197669, -0.65197669,  0.15786537, -0.63017953],
       [ 1.42438169,  0.        ,  0.        , -2.70653792, -1.28215623],
       [ 1.42438169,  0.        ,  0.        , -2.70653792, -1.28215623],
       [ 0.15786537, -0.63017953, -0.63017953, -1.4400216 , -0.65197669],
       [-2.70653792, -1.28215623, -1.28215623,  1.42438169,  0.        ]])

What if we want to change the stride? That's just how much we're updating m and n in the inner loop and h and k in the outer loop. (You can also see how we might generalize to different horizontal and vertical strides, but we're not going to worry about that; no one really does that I don't think.) We'll go with a stride of 3, since that works for our 5x5 image with 2x2 filters (even though we'll lose information about the middle of our image - oh no! But then, we shouldn't be using a one-layer neural network to classify a single 5x5 array of 1-bit pixels, so no need to hand out the speeding ticket when there's a murder weapon in the trunk).

Note that our horizontal and vertical ranges are simply the formulas for computing the volume of a convolution layer (or area of a single filter applied to the input image). We assume $F=2$, since we're applying a 2x2 filter; and $P=0$, since we're not applying any padding. We already know that $W_{1} = W_{2} = 5$. Thus, $i$ and $j$ loop over range $(5-2)/3 + 1 = 3/3 + 1 = 1 + 1 = 2$. We'll also need to convert the output of our division from float to integer with `int` (can't loop over a float). Let's give it a go:

In [136]:
def conv_2x2(X,s):
    W=np.random.randn(2,2)
    f=int((5-2)/s)+1
    V=[]
    h=0
    k=2
    m=0
    n=2
    for i in range(f):
        for j in range(f):
            V.append(sum(sum(W*X[h:k,m:n])))
            m+=s
            n+=s
        h+=s
        k+=s
        m=0
        n=2
    V=np.asarray(V)
    V=V.reshape(f,f)
    return V

What happens when we apply a 2x2 convolution with a stride of 3 to our input?

In [137]:
conv_2x2(X,3)

array([[-1.03324359,  0.90379039],
       [ 0.90379039, -1.03324359]])

Great! How about we add padding? There's actually a built-in function called `np.pad` that'll add zeros (by default) to an array. We can do this with `mode='constant'` and `pad_width=1` and store the result to `X` to update our input array (image) with zero padding of 1. We can thus set `pad_width=p` for generic zero padding. How do we implement this?

In [138]:
def conv_2x2(X,s,p):
    X=np.pad(X, mode='constant', pad_width=p)
    W=np.random.randn(2,2)
    f=int((5-2+2*p)/s)+1
    V=[]
    h=0
    k=2
    m=0
    n=2
    for i in range(f):
        for j in range(f):
            V.append(sum(sum(W*X[h:k,m:n])))
            m+=s
            n+=s
        h+=s
        k+=s
        m=0
        n=2
    V=np.asarray(V)
    V=V.reshape(f,f)
    return V

Now let's try a 2x2 convolution with our input array $X$, stride of 1, and one "layer" of zero padding:

In [139]:
conv_2x2(X=X,s=1,p=1)

array([[ 0.        ,  1.29846016,  0.55032169,  0.55032169, -0.74813847,
         0.        ],
       [ 1.29846016, -0.576771  ,  0.15978072,  0.15978072,  1.28687341,
        -0.74813847],
       [ 1.46982763, -0.75972522,  0.        ,  0.        ,  1.46982763,
        -0.75972522],
       [ 1.46982763, -0.75972522,  0.        ,  0.        ,  1.46982763,
        -0.75972522],
       [ 0.17136747,  1.28687341,  0.55032169,  0.55032169, -0.576771  ,
        -0.01158675],
       [ 0.        ,  0.17136747,  0.15978072,  0.15978072, -0.01158675,
         0.        ]])

We can also do the same thing we did with our 1x1 convolution: repeat this operation `K` times for a volume of `K` outputs. This means we'll need `K` filters ${W_{1}, ... , W_{K}}$. In this notation, we use big $K$ since little $k$ is being used to index our array:

In [140]:
def conv_1x1(X, s, K):
    V = []
    if s > 0:
        w = np.random.randn(1,K)
        [V.append(X[0:len(X):s]*w[0][i]) for i in range(k)]
        return V
    else: 
        return "you can't do that"

How would we extend this to a 2x2 kernel?

In [141]:
def conv_2x2(X,s,p,K):
    X=np.pad(X, mode='constant', pad_width=p)
    W=np.random.randn(K,2,2)
    f=int((5-2+2*p)/s)+1
    V=[]
    h=0
    k=2
    m=0
    n=2
    for i in range(f):
        for j in range(f):
            [V.append(sum(sum(W[0]*X[h:k,m:n]))) for q in range(K)]
            m+=s
            n+=s
        h+=s
        k+=s
        m=0
        n=2
    V=np.asarray(V)
    V=V.reshape(K,f,f)
    return V

What happens when we pass four outputs to our 2x2 convolution with a stride of 1 and padding of 1?

In [142]:
conv_2x2(X,s=1,p=1,K=4)

array([[[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.76887614,  0.76887614],
        [ 0.76887614,  0.76887614, -0.59511626, -0.59511626,
         -0.59511626, -0.59511626],
        [-0.59511626, -0.59511626, -0.59511626, -0.59511626,
         -1.36399239, -1.36399239],
        [-1.36399239, -1.36399239,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.76887614,  0.76887614,  0.76887614,  0.76887614,
         -2.1384605 , -2.1384605 ],
        [-2.1384605 , -2.1384605 ,  1.05424433,  1.05424433,
          1.05424433,  1.05424433]],

       [[ 1.05424433,  1.05424433,  1.05424433,  1.05424433,
          2.59758857,  2.59758857],
        [ 2.59758857,  2.59758857, -1.36399239, -1.36399239,
         -1.36399239, -1.36399239],
        [-0.00559197, -0.00559197, -0.00559197, -0.00559197,
          0.46472004,  0.46472004],
        [ 0.46472004,  0.46472004,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.   

Last step! We want to generalize to any square convolution. We'll use $H$ to denote the height (and width, but $W$ is already our weights matrix). Furthermore, we'll replace that hard-coded 5 so that we can convolve on images of other sizes. We won't even assume they're square images!

In [143]:
def conv_2d(X,H,s,p,K):
    F=X.shape[1]
    G=X.shape[0]
    X=np.pad(X, mode='constant', pad_width=p)
    W=np.random.randn(K,H,H)
    f=int((F-H+2*p)/s)+1
    g=int((G-H+2*p)/s)+1
    V=[]
    h=0
    k=H
    m=0
    n=H
    for i in range(g):
        for j in range(f):
            [V.append(sum(sum(W[0]*X[h:k,m:n]))) for q in range(K)]
            m+=s
            n+=s
        h+=s
        k+=s
        m=0
        n=H
    V=np.asarray(V)
    V=V.reshape(K,f,g)
    return V

I'm sure this code can be cleaner, but let's just see if it works on a random 6x8 image with 2-bit pixels. `np.random.randint` is our friend here. First we'll generate our image. We'll call it $X$ again:

In [144]:
X = np.random.randint(0,4,48)
X = X.reshape(6,8)
X

array([[0, 0, 1, 0, 1, 2, 3, 2],
       [1, 3, 0, 3, 3, 0, 0, 1],
       [0, 3, 2, 0, 1, 0, 3, 1],
       [0, 3, 2, 0, 3, 0, 1, 0],
       [3, 2, 3, 3, 1, 1, 1, 3],
       [0, 1, 2, 2, 2, 3, 2, 0]])

We'll use a 2x2 kernel with stride of 1 and padding of 1. Don't ask why we're adding padding with a stride of 1 lol. Actually, do ask: it means you get what's going on! We'll generate 4 output volumes:

In [145]:
conv_2d(X=X,H=2,s=1,p=1,K=4)

array([[[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  2.86716489,  2.86716489,  2.86716489,
          2.86716489, -0.27195007, -0.27195007],
        [-0.27195007, -0.27195007,  2.86716489,  2.86716489,
          2.86716489,  2.86716489,  5.46237971],
        [ 5.46237971,  5.46237971,  5.46237971,  8.05759454,
          8.05759454,  8.05759454,  8.05759454],
        [ 4.91847957,  4.91847957,  4.91847957,  4.91847957,
         -0.54390014, -0.54390014, -0.54390014],
        [-0.54390014,  2.86716489,  2.86716489,  2.86716489,
          2.86716489,  8.32954461,  8.32954461],
        [ 8.32954461,  8.32954461, -0.27708113, -0.27708113,
         -0.27708113, -0.27708113,  9.93084612],
        [ 9.93084612,  9.93084612,  9.93084612,  8.32441355,
          8.32441355,  8.32441355,  8.32441355],
        [ 1.5910394 ,  1.5910394 ,  1.5910394 ,  1.5910394 ,
          4.27501013,  4.27501013,  4.27501013]],

       [

Yes! We got something that resembles a convolution. Now let's go back to our 5x5 1-bit image example. Suppose we were to classify our image by applying a 2x2 convolution with stride of 1 and zero padding (zero zero padding?) Remember padding doesn't make too much sense with a stride of 1. We'll always apply our convolution to every pixel with a stride of 1. Let's also learn eight sets of shared weights (that's our "output" or "volume"). How many parameters do we learn?

Let's see. It's 2x2=4 parameters for each filter/kernel/weight matrix, and we're going to learn 8 of those. So 2x2x8=32 parameters. That's not that many compared to what we had before (10x25+10=250+10=260 parameters for a 10x25 weight matrix and a bias vector of length 10). We could optionally add a bias to our convolutional layer (I'll leave this as an exercise for the reader. It's not too hard!) On the other hand, we've done a bit more work to compute our output for each filter, since we've had to stride the kernel across the entire array of pixels. 

But we can't possibly be done. We still need to output a probability vector, or at the very least some vector of length 10. We could simply use a fully-connected layer, or linear transformation to do this: flatten the output of our convolutional layer into a vector of length 32, multiply a 10x32 matrix by the vector, and we'll have our output vector of length 10! Pass that through a softmax and we're good to go (still need to backprop, but that's just calculus).

However, there's another thing that the geniuses behind deep learning came up with, and it's called **"max pooling"**. How this works is you just take the maximum value for a specified area (receptive field) of each volume produced by a particular dot product of kernel and input, and then stride over the volume. As you stride horizontally, you output the maximum value into the first row of a new **downsampled** matrix. Once you reach the end of the input **feature map** (the output volumes of our convolutional layer are input feature maps to our pooling layer), you then stride vertically. Just like you're doing a convolution! I think this will make a lot more sense with an example.

Let's take our 5x5 "zero array" image and apply our 2x2 convolution with stride 1 and output 4: 

In [146]:
conv_output = conv_2d(X=a_zero_array,H=2,s=1,p=0,K=4)
conv_output

array([[[-2.45320071, -2.45320071, -2.45320071, -2.45320071],
        [-1.59941187, -1.59941187, -1.59941187, -1.59941187],
        [-1.59941187, -1.59941187, -1.59941187, -1.59941187],
        [-0.84545581, -0.84545581, -0.84545581, -0.84545581]],

       [[-1.17478965, -1.17478965, -1.17478965, -1.17478965],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [-2.12386687, -2.12386687, -2.12386687, -2.12386687]],

       [[-1.17478965, -1.17478965, -1.17478965, -1.17478965],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [-2.12386687, -2.12386687, -2.12386687, -2.12386687]],

       [[-0.84545581, -0.84545581, -0.84545581, -0.84545581],
        [-1.69924465, -1.69924465, -1.69924465, -1.69924465],
        [-1.69924465, -1.69924465, -1.69924465, -1.69924465],
        [-2.45320071, -2.45320071, -2.45320071, -2.45320071]]])

Let's take a look at how our convolutional layer changed the dimensions of our input image:

In [147]:
print(a_zero_array.shape)
print(conv_output.shape)

(5, 5)
(4, 4, 4)


Wow, we barely downsampled our image! That makes sense: we're applying small (2x2) filters, and we're only striding one pixel at a time, so we're reusing each pixel multiple times. With larger filter sizes, typically used on "real" images with many more pixels, we tend to downsample our images a good deal. Even with 3x3 filters and a stride of 2 (a common operation), you can imagine how this would work. Actually, we can try this out on our 5x5 image. Let's output four filters:

In [148]:
a_zero_array.shape

(5, 5)

In [149]:
conv_output2 = conv_2d(X=a_zero_array,H=3,s=2,p=0,K=4)
print(conv_output2)
print(conv_output2.shape)

[[[ 1.43149484  1.43149484]
  [ 1.43149484  1.43149484]]

 [[-0.35393574 -0.35393574]
  [-0.35393574 -0.35393574]]

 [[ 1.50252952  1.50252952]
  [ 1.50252952  1.50252952]]

 [[ 0.84623843  0.84623843]
  [ 0.84623843  0.84623843]]]
(4, 2, 2)


I've been listing our output first, which is kinda unconventional, but you get the idea what's going on here. We downsample the area of the input array from 5x5 to 2x2. At the same time, we increase the depth of our image from 1 to 4. In total, we go from 5x5=25 pixels to 2x2x4=16 weights.

Now let's return to our 5x5 array. Here we went from 5x5=25 pixels to 4x4x4=64 weights! How do we downsample? Let's try pooling. Our first dimension is the output volume. We can extract a single volume:

In [150]:
conv_output[0]

array([[-2.45320071, -2.45320071, -2.45320071, -2.45320071],
       [-1.59941187, -1.59941187, -1.59941187, -1.59941187],
       [-1.59941187, -1.59941187, -1.59941187, -1.59941187],
       [-0.84545581, -0.84545581, -0.84545581, -0.84545581]])

Now we can define our pooling operation. It's the same as our convolution: we have a (typically square) shape (let's say 2x2) and a stride (let's say 2). Let $C$ be the output of our convolutional layer, $H$ be the height and width of the pooling region, and $s$ be our stride.

For a square pooling region accepting a volume with equal width and height $W_{2} = H_{2}$, the pooling width and height $W_{3} = H_{3} = (W_{2} - F)/s + 1$, where $F$ is the width and height of the pooling region and $s$ is the pooling stride. The pooling depth $D_{3}$ is equal to the volume depth $D_{2}$.

NumPy has an operation `np.amax` that returns the maximum value of an ndarray. 

In [151]:
def pool_2x2(C,F,s):
    H=int((C.shape[1]-F)/2+1)
    P=[]
    h=0
    k=F
    m=0
    n=F
    for q in range(C.shape[0]):
        for i in range(H):
            for j in range(H):
                P.append(np.amax(C[q][h:k,m:n]))
                m+=s
                n+=s
            h+=s
            k+=s
            m=0
            n=2
        h=0
        k=F
        m=0
        n=F
    P=np.asarray(P)
    P=P.reshape(C.shape[0],F,F)
    return P

Now let's examine the output of our pooling operation:

In [152]:
P = pool_2x2(C=conv_output,F=2,s=2)
P

array([[[-1.59941187, -1.59941187],
        [-0.84545581, -0.84545581]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[-0.84545581, -0.84545581],
        [-1.69924465, -1.69924465]]])

We've also downsampled further, cutting our width and height in half while retaining a depth of 4. Again, it's not conventional to list the depth first, but it prints nice and works out well with the list comprehension in the convolutional layer (okay, I could have just added a third for loop like I did with the pooling layer):

In [153]:
print(P.shape)

(4, 2, 2)


We still need to get from our pooling layer to our output layer, and finally to our softmax layer. Really at any time we can just flatten our output into a vector of length $m$ and multiply an $nxm$ matrix by this vector to get a vector of length $n$. That's essentially what we'll do, but we still have one more operation to add. We'll get to it soon.

All we've covered so far should also make us appreciate how easy it is to construct convolutional and pooling layers in PyTorch. Recall our old neural network:

In [154]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(25, 10)
    def forward(self, x):
        return self.fc1(x)

We can now replace our fully-connected (or **dense**) layer for our 2x2 convolutional layer with stride of 1 and volume of 4! PyTorch's nn module makes this super easy with `nn.conv2d`. `2d` refers to the dimensions of the kernel. We don't even have to define the kernel size and stride (though we can). We simply pass our number of input channels `in_channels` (we only have 1 channel for our single image of 1-bit pixels), `out_channels` (our volume of 4), `kernel_size` (2, PyTorch assumes square kernels by default) and `stride` (2, with horizontal and vertical stride equal by default). By convention, we call our first convolutional layer `conv`:

In [155]:
conv = nn.Conv2d(in_channels=1,out_channels=4,kernel_size=2,stride=2)

Alternatively, we can just pass the values of the arguments:

In [156]:
conv = nn.Conv2d(in_channels=1,out_channels=4,kernel_size=2,stride=2)
print(conv)
conv2 = nn.Conv2d(1,4,2,2)
print(conv)

Conv2d(1, 4, kernel_size=(2, 2), stride=(2, 2))
Conv2d(1, 4, kernel_size=(2, 2), stride=(2, 2))


We can examine our weights with `conv.weight`:

In [157]:
conv.weight

Parameter containing:
tensor([[[[-0.1626, -0.4021],
          [-0.0743, -0.0409]]],


        [[[ 0.0948, -0.4035],
          [-0.1412,  0.1519]]],


        [[[-0.4209,  0.3761],
          [ 0.0372, -0.3109]]],


        [[[-0.3881, -0.1476],
          [ 0.4934,  0.0552]]]], requires_grad=True)

PyTorch also adds a bias by default. You can specify `bias=false` if you don't want this, but why not add a bias? Biases are computationally cheap (if that matters when doing matrix math on a 5x5 image of one-bit pixels). We can examine them using `conv.bias`:

In [158]:
conv.bias

Parameter containing:
tensor([-0.0112, -0.4430, -0.4955, -0.2481], requires_grad=True)

Now we can swap in our `conv` for `fc` in our `init` and `forward` modules:

In [159]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(1,4,2,2)
    def forward(self, x):
        return self.conv(x)

Examining a `net` instance of our `Net()`, we see that we have one two-dimensional convolution layer with a batch size of 1, output volume of 4, and 5x5 kernels with a stride of 2 in both directions:

In [160]:
net = Net()
net

Net(
  (conv): Conv2d(1, 4, kernel_size=(2, 2), stride=(2, 2))
)

Now let's add our pooling layer `nn.MaxPool2d`. We pass (2,2) for the size and stride of our receptive field. By convention, we'll call this `pool`:

In [161]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(1,4,2,2)
        self.pool = nn.MaxPool2d(2,2)
    def forward(self, x):
        return self.pool(self.conv(x))

Let's check out our new **multilayer** (deep??) neural network:

In [162]:
net = Net()
net

Net(
  (conv): Conv2d(1, 4, kernel_size=(2, 2), stride=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

Looks like `MaxPool2d` has a few other default parameters. You might recognize `padding` from our convolution operation. We can also pad our input with zeros for pooling. For `padding=N`, `N` is just the size of the padding (how many zero cells extending horizontally or vertically from the original border of the image).

`dilation` refers to spacing between values in a convolutional or padding kernel. For instance, if we applied our 2x2 kernel with a `dilation rate` of 2, we would end up applying our operation over a 3x3 field but only performing elementwise multiplication on every other row or column.

A visualization DEFINITELY helps to explain dilation (and other convolutional operations!) https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d

What about `ceil_mode`? Remember when we computed the dimensions of the output of our convolutional layer? We can do the same for the output of our pooling layer. We don't need to go over the details of the operation (see https://pytorch.org/docs/stable/\_modules/torch/nn/modules/pooling.html if you're curious). The point is that our dimensions must be (positive) integers, and thus if the operation produces a non-integer output, we'll either need to round down (floor) or up (ceiling). When `ceil_mode=False` (by default), PyTorch uses the floor of the operation rather than the ceiling to compute the dimensions of the pooling output.

Before we add our fully connected and softmax layers, we're going to introduce one final operation. When we "do" "**deep learning"** (deep learn?), we're stacking multiple layers of (frequently) linear operations. But we also want to introduce nonlinear operations so that our neural network "function" can better approximate nonlinear relationships between our inputs and outputs. There are other good reasons for introducing nonlinearities, but let's save the theory for another time.

The particular class of nonlinear operations we'll examine here are called **activation functions**. If we think back to our conceptualization of a neural network as a set of nodes organized into layers and connected via (fully connected, convolutional, and other) operations, we're (crudely) modeling a neurological process whereby input data (axons) are transmitted into outputs. But what tells us whether (or how) our neuron will "fire" or "activate"? Here's where activation functions come in.

Technically, activations don't need to be nonlinear. Let's define a random input to our activation function (a vector of length 8):

In [163]:
X = np.random.randn(8)
X

array([-1.05561361, -0.68337371, -0.25888077,  0.15508642,  1.63016968,
       -0.61440984, -0.04938359, -1.04421923])

We'll now define a simple (hypothetical, pointless) activation function `identity` that returns its input:

In [164]:
def identity(X):
    return X

It should be pretty obvious what's going to happen if we pass $X$ to the function:

In [165]:
identity(X)

array([-1.05561361, -0.68337371, -0.25888077,  0.15508642,  1.63016968,
       -0.61440984, -0.04938359, -1.04421923])

Let's say we want to go a "step" further and return 0 if $x_{i}$ is below a certain threshold (say, 0) and 1 if $x_{i}$ is at or above the threshold, for each $x_{i}$ in vector $X$. For fun, you could generalize this to a generic tensor.

In [166]:
def step(X):
    for i in range(len(X)):
        if X[i] < 0:
            X[i] = 0
        else:
            X[i] = 1
    return X

What's the output of this function for a random vector `X` (length 8). Let's print `X` first for comparison, then apply `step`:

In [167]:
X = np.random.randn(8)
print(X)
step(X)

[-0.66621633 -1.72723714  0.31375181  0.8125176  -0.73874976  0.90937177
 -1.02299779 -0.6335149 ]


array([0., 0., 1., 1., 0., 1., 0., 0.])

However, we've got a few issues with our step function. It's not smooth, it's coarse, and most problematically it's derivative is zero at all points. That means when we backprop we're going to kill all the gradients. No good.

But what if we combine these two completely useless activation functions: the one (identity function) that doesn't update outputs during the forward pass with the one (step function) that doesn't update weights during the backward pass? Now we have the (no joke) go-to activation function for convolutional neural networks: the **rectified linear unit (ReLU)**, or less pretentiously, the "thing that sets our negative values to zero":

In [168]:
def ReLU(X):
    for i in range(len(X)):
        if X[i] < 0:
            X[i] = 0
        else:
            X[i] = X[i]
    return X

Let's create another random vector `X` and apply `ReLU` to it:

In [169]:
X = np.random.randn(8)
print(X)
ReLU(X)

[-1.02432329 -0.42662539 -0.24859638  0.59464189  1.03328345  0.6536854
 -1.49163869 -1.25539448]


array([0.        , 0.        , 0.        , 0.59464189, 1.03328345,
       0.6536854 , 0.        , 0.        ])

Nice! Let's add this to our network architecture. Of course PyTorch's functional API (which we imported as `F`) has a built-in function for ReLU, `F.ReLU`. Let's add it to our `Net()`:

In [170]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(1,4,2,2)
        self.pool = nn.MaxPool2d(2,2)
    def forward(self, x):
        return self.pool(F.relu(self.conv(x)))

Before we add our softmax and loss function and define our backpropagation, we still need a fully-connected layer to transform the output of `self.pool(F.relu(self.conv(x)))` to a vector of length 10 (again, the number of classes).

Let's review how the dimensions of our image have changed as the image is passed through the network. We began with a 5x5 image of 1-bit pixels and converted it to a PyTorch tensor `X`. We called the `float()` method on the model to convert the model to a float. Let's do this again, since our `X` is probably something else by now:

In [171]:
X = torch.tensor(a_zero_array, requires_grad=False).float()
print(X.shape)
X

torch.Size([5, 5])


tensor([[0., 1., 1., 1., 0.],
        [1., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1.],
        [0., 1., 1., 1., 0.]])

Next we apply four 2x2 convolutions with stride of 1. Before we pass `X` to `nn.Conv2d`, we need to convert the tensor to size \[batch_size, channels, height, width\]. Right now it's of size \[5,5\]. `view` does the trick, adding dimensions for batch_size (just our 1 image) and input_channels (the one "color channel" of 1-bit pixels). Let's apply the `view` method to `X`:

In [172]:
X = X.view(1, 1, 5, 5)
print(X.shape)
X

torch.Size([1, 1, 5, 5])


tensor([[[[0., 1., 1., 1., 0.],
          [1., 0., 0., 0., 1.],
          [1., 0., 0., 0., 1.],
          [1., 0., 0., 0., 1.],
          [0., 1., 1., 1., 0.]]]])

Now let's apply `nn.Conv2d` to `X`. We'll go with the 1 input channel, 4 output channels, a kernel size of 2, and a stride of 2. Let's set the output to `C`, print its shape, and call it:

In [173]:
conv = nn.Conv2d(1,4,2,2)
C = conv(X)
print(C.shape)
C

torch.Size([1, 4, 2, 2])


tensor([[[[-0.1791, -0.7776],
          [-0.8830, -0.4954]],

         [[-0.3681, -0.3685],
          [-0.5546,  0.4276]],

         [[ 0.3045,  0.2550],
          [ 0.0035,  0.4882]],

         [[-0.3303,  0.1250],
          [-0.1404, -0.4544]]]], grad_fn=<MkldnnConvolutionBackward>)

How does our pooling operation change the shape? What happens when we apply two-dimensional max pooling with a stride of 2?

In [174]:
pool = nn.MaxPool2d(2,2)
P = pool(C)
print(P.shape)
P

torch.Size([1, 4, 1, 1])


tensor([[[[-0.1791]],

         [[ 0.4276]],

         [[ 0.4882]],

         [[ 0.1250]]]], grad_fn=<MaxPool2DWithIndicesBackward>)

We see now that we've downsampled our input so much that we actually have **fewer** nodes than classes! We need a linear transformation that'll get us from four nodes to 10. Sounds like a job for a 10x4 matrix, with a(n optional) 10x1 vector!

Before we do this, we need to **flatten** the output of our pooling operation using `view`. We can hard code this as `P.view(4)`, since the number of total features should be clear:

In [175]:
P = P.view(4)
P.shape
P

tensor([-0.1791,  0.4276,  0.4882,  0.1250], grad_fn=<ViewBackward>)

Remember in PyTorch, we don't need to specify the dimensions of a weights matrix and bias vector. We can just call `nn.linear` with `use_bias=True` and specify our number of input nodes (4) and output nodes (10). We apply this to the reshaped output of the the convolution, relu, and pooling operation. Since we're already passing the output of our fully-connected (**dense**) layer through a softmax function, we don't need an additional ReLU activation. What does our updated network look like?

In [176]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(1,4,2,2)
        self.pool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(4,10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv(x)))
        x = x.view(-1, 4)
        x = self.fc(x)
        return x

As you may have noticed, printing our `net` instance of `Net()` only displays the layers defined in our `__init__` constructor. Let's do this!

In [177]:
net = Net()
print(net)

Net(
  (conv): Conv2d(1, 4, kernel_size=(2, 2), stride=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=4, out_features=10, bias=True)
)


Remember, we don't need to apply our cross-entropy loss to a softmax transformation of our output. `F.cross_entropy` will take care of the softmax for you. Since we're working in `nn` (and I'm using this as a reference: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html), we'll use the `nn` implementation of cross-entropy: `nn.CrossEntropyLoss()`:

In [178]:
ce_loss = nn.CrossEntropyLoss()

We also need an **optimizer**. It turns out that "vanilla" gradient descent (where we subtract the scaled partial derivatives of our loss function with respect to each of our weights and biases) for each of our training samples isn't usually implemented in practice (horror face emoji). Instead, we use something called **stochastic gradient descent (SGD)**. Unlike "vanilla" or **batch gradient descent**, which calculates the gradient using the entire training dataset before updating weights, stochastic gradient descent updates the gradient using data from one observation at a time. Each observation is randomly selected from the dataset, hence "stochastic."

Of course, none of this matters when we're feeding a single observation through the network. But we'll use SGD because it tends to work better than batch gradient descent for large datasets. Hey if we were selecting an optimal algorithm for classifying a single 5x5 image of one-bit pixels, we probably wouldn't use a neural network. (We'd probably just give up...)

PyTorch has an optimizer package called `torch.optim`. If you really want to nerd out on this stuff, there are a ton of optimizers, and many of them are in this package. If you know one that isn't, you could contribute it to PyTorch! A lot of these optimizers add some form of momentum to prevent the network from getting stuck in a local minimum. We won't worry about that for now. Let's import our optimizer package:

In [179]:
import torch.optim as optim

Now let's define our optimzer using `optim.SGD`. First we'll need to pass our `net.parameters()` to the optimizer. This is just our weights and biases (prior to updating):

In [180]:
net.parameters()

<generator object Module.parameters at 0x112567cf0>

Next, we'll add our learning rate, `lr`. We can stick with `lr=0.1`. We'll leave out **momentum** for now, but you can experiment with adding this argument.

PyTorch classifier tutorial: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py

torch.optim.sgd source code: https://pytorch.org/docs/stable/\_modules/torch/optim/sgd.html

An incredible (and comprehensive) summary of optimization algorithms: http://ruder.io/optimizing-gradient-descent/

Let's define our optimizer!

In [181]:
optimizer = optim.SGD(net.parameters(), lr=0.1)

What happens if we feed our image into our network?

In [182]:
output = net(X)
output

tensor([[-0.3231, -0.0458,  0.0932, -0.3140, -0.2863, -0.1747, -0.4393, -0.4582,
          0.3943, -0.3031]], grad_fn=<AddmmBackward>)

Remember, we apply cross-entropy loss on our "raw" output (not passed through the softmax function). However, we can still apply `F.softmax` to our raw output to view our probability vector (tensor). What do we get here?

In [183]:
predictions = F.softmax(output)
predictions

/Users/pp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[0.0842, 0.1111, 0.1277, 0.0850, 0.0874, 0.0977, 0.0750, 0.0736, 0.1725,
         0.0859]], grad_fn=<SoftmaxBackward>)

In our example, the index will be equivalent to the value of the digit, so we can just apply `torch.argmax` (which gives us the index of our maximum value) to our "probability tensor":

In [184]:
torch.argmax(predictions)

tensor(8)

We see that the network predicts a "5." Remember, our ground truth is a "0." This isn't a big deal. We haven't updated our weights and biases yet, so this is entirely random. Before we update our parameters (using this one example!), let's compute our loss (on our `output`, prior to applying softmax):

In [185]:
loss = ce_loss(output, y)
loss

tensor(2.4745, grad_fn=<NllLossBackward>)

We can confirm that this is the negative natural logarithm of the predicted output for the zero index. As with other NumPy operations, we can just swap `torch.log` for `np.log`:

In [186]:
-torch.log(predictions[0][0])

tensor(2.4745, grad_fn=<NegBackward>)

Let's compute our gradients. We pass the argument `retain_graph=True` so that we can reuse this method:

In [187]:
loss.backward(retain_graph=True)

We update parameters by passing the `step()` method to our optimizer:

In [188]:
optimizer.step()

Let's do another forward pass of our "data":

In [189]:
output = net(X)
output

tensor([[-0.1271, -0.0962,  0.0215, -0.3068, -0.3658, -0.2317, -0.4788, -0.4921,
          0.3598, -0.3862]], grad_fn=<AddmmBackward>)

We can see that our loss has decreased:

In [190]:
loss = ce_loss(output, y)
loss

tensor(2.2526, grad_fn=<NllLossBackward>)

Maybe it's even getting closer to the prediction...?

In [191]:
torch.argmax(predictions)

tensor(8)

Of course in the "real world" we train neural networks with many more parameters, on observations with many more features, on datasets with many more observations. 

Hopefully what our example has done is to enable you to learn the **mechanics** of training a neural network. With a single 5x5 array (or tensor) of binary pixels, it is easier to wrap one's head around the process of learning weights and biases through matrix multiplications and additions, convolutions, activations, and softmax functions.

Nevertheless, there's a lot that we haven't covered. We didn't really go over the practical issues involved in training a neural network with real data: stuff like data ingestion, resizing and augmenting data, partitioning our data into training and test sets, and even simple things like writing print statements to record our loss or outputting our trained model files at different checkpoints.

Let's recap what we've learned!

We began by exploring the simple case of linear regression, first with a single weight, and then with a weight and a bias. While we can use a neural network to learn these parameters, it usually makes more sense to solve for our parameters analytically.

From linear regression, we moved to using a **fully-connected** network (matrix multiplication and addition) to transform our input into output. We examined the logic of **forward propagation** (computing output and loss) and **backward propagation** (or **backpropagation**: updating weights and biases) using NumPy.

Although NumPy operations are great for learning the logic of training a neural network, we can save a lot of time by using PyTorch. Let's conclude by reviewing the simple network we constructed. 

We define a `Net` class comprised of a **network architecture** (`__init__`) and our forward propagation, or **forward pass** (`forward`):

In [192]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(1,4,2,2)
        self.pool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(4,10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv(x)))
        x = x.view(-1, 4)
        x = self.fc(x)
        return x

Our architecture consists of three **layers**:

1) A **convolutional layer**, which slides a matrix (filter) of shared weights across the input, computes the dot product at each point, and repeats the operation for an arbitrary number of filters.

2) A **pooling layer**, which computes the maximum value of each input over a specified region.

3) A **fully-connected (dense) layer**, which performs a linear transformation (matrix multiplication and addition) on the input.

We then define three operations in our **forward** pass:

1) A "conv-relu-pool" triple whammy, whereby we pass `x` through the convolutional layer, apply our `ReLU` activation, and max pool the output.

2) A "flatten" operation, `x.view`, whereby we reshape our output into a one-dimensional tensor (vector?) or length 4.

3) A "linear" operation, whereby we "upsample" from our flattened output to a "class vector" of length 10.

Now we can compute our output...

In [193]:
net = Net()
print(net)
output = net(X)
print(output)

Net(
  (conv): Conv2d(1, 4, kernel_size=(2, 2), stride=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=4, out_features=10, bias=True)
)
tensor([[-0.3053,  0.1836, -0.4118,  0.4404,  0.1585,  0.0436, -0.2452, -0.3173,
         -0.5120,  0.1199]], grad_fn=<AddmmBackward>)


...display our predicted probabilities and predicted class:

In [194]:
predictions = F.softmax(output)
print(predictions)
prediction = torch.argmax(predictions)
print(prediction)

tensor([[0.0767, 0.1251, 0.0690, 0.1617, 0.1220, 0.1087, 0.0815, 0.0758, 0.0624,
         0.1173]], grad_fn=<SoftmaxBackward>)
tensor(3)


/Users/pp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


...calculate our loss:

In [195]:
optimizer = optim.SGD(net.parameters(), lr=0.1)
ce_loss = nn.CrossEntropyLoss()
loss = ce_loss(output, y)
loss.backward()

..and backpropagate:

In [196]:
optimizer.step()

We've covered a lot for an afternoon. Hopefully you now understand a bit more about what's going on when you train a neural network. Once you get the logic behind the training process, the rest is just engineering...

Please send feedback to donnelly.patrick.t@gmail.com!